In [73]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pickle
import subprocess
from bs4 import BeautifulSoup
import time
import json

# 1. 打开Chromedriver

In [10]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-gpu")

# 设置 ChromeDriver 路径
chrome_driver_path = ".twiiter/chromedriver-mac-arm64/chromedriver"
service = Service(chrome_driver_path)

In [66]:
driver = webdriver.Chrome(service=service, options=chrome_options)

In [67]:
driver.get("https://x.com") #这里可能会让你登陆，不要管！！

# 2. 写入小饼干

In [62]:
#获取小饼干，只需运行一次
# cookies = driver.get_cookies()
# with open("twitter_cookies.pkl", "wb") as file:
#     pickle.dump(cookies, file)

In [69]:
with open("twitter_cookies.pkl", "rb") as file:
    cookies = pickle.load(file)

for cookie in cookies:
    driver.add_cookie(cookie)

# 3. 输入目标网址（例如OrbitalSidekick）

In [70]:
url = "https://x.com/OrbitalSidekick" #这里改成你要爬的主页
driver.get(url)

# 4. 开爬

In [71]:
scraped_tweets = set()
scroll_times = 5 #twitter的反爬，一次只能爬一丢丢，所以每轮滑动五下爬第二次
max_batches = 5 #设置一下轮数（自己感觉一下，大致需要多少轮的文本）

for batch in range(1, max_batches + 1):
    print(f"开始爬取第 {batch} 轮推文.")

    for _ in range(scroll_times):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    tweets = soup.find_all("div", {'data-testid': "cellInnerDiv"})

    for tweet in tweets:
        content_div = tweet.find('div', {'data-testid': "tweetText"})
        if content_div:
            tweet_text = " ".join(content_div.stripped_strings)

            if tweet_text not in scraped_tweets:
                scraped_tweets.add(tweet_text)
                print(f"推文 {len(scraped_tweets)}: {tweet_text}\n" + "-"*80)

    print(f"结束第 {batch} 轮爬取，共爬取 {len(scraped_tweets)} 条推文。\n")

driver.quit()

开始爬取第 1 轮推文.
推文 1: Our advanced satellite technology goes beyond high-resolution imagery- it analyzes the invisible spectrum, revealing methane emissions! On 5/21/24, our GHOSt2 satellite found a methane plume in CA in less than 24 hrs! #esg #hyperspectral #methane
--------------------------------------------------------------------------------
推文 2: OSK satellites are methane leak ninjas! 
We monitor pipelines & just spotted a leak in Turkmenistan...from SPACE! Data analyzed & leak detected in 48 hours! #hyperspectral ”
--------------------------------------------------------------------------------
推文 3: We’re proud to announce that we are a Qualified Team in the #XPRIZEWildfire competition! We are helping to put an end to destructive wildfires through our technology within the Space-Based Wildfire Detection & Intelligence track. Stay tuned for our journey to the semifinals!
--------------------------------------------------------------------------------
推文 4: Orbital Sidekick shares

# 5. 保存Json文件

In [ ]:
with open('tweets.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(list(scraped_tweets), jsonfile, ensure_ascii=False, indent=4)
print("DONE!")

In [ ]:
# !pipreqs . --force

INFO: Not scanning for jupyter notebooks.
INFO: Successfully saved requirements file in ./requirements.txt
